In [1]:
!pip install darts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.9/846.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.8/279.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB

In [2]:
from darts import TimeSeries
import pandas as pd

In [3]:
def split_train_test(df,max_date_train,min_date_test):
  treino = df.query("Data <= @max_date_train")
  teste= df.query("Data >= @min_date_test")
  return treino,teste

In [4]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

def calculate_metrics(observed_values, predicted_values):
    # RMSE (Root Mean Squared Error)
    rmse = np.sqrt(mean_squared_error(observed_values, predicted_values))

    # MSE (Mean Squared Error)
    mse = mean_squared_error(observed_values, predicted_values)

    # MAE (Mean Absolute Error)
    mae = mean_absolute_error(observed_values, predicted_values)

    # MAPE (Mean Absolute Percentage Error)
    def mean_absolute_percentage_error(observed_values, predicted_values):
        return np.mean(np.abs((observed_values - predicted_values) / observed_values)) * 100
    mape = mean_absolute_percentage_error(observed_values, predicted_values)

    # R² (Coeficiente de Determinação)
    r2 = r2_score(observed_values, predicted_values)

    # MASE (Mean Absolute Scaled Error) - Necessário calcular os erros do modelo de benchmark
    naive_forecast = np.roll(observed_values, 1)  # Utilizando previsão ingênua (shift de 1)
    naive_errors = np.abs(observed_values - naive_forecast)
    mase = np.mean(np.abs(observed_values - predicted_values) / naive_errors)

    # sMAPE (Symmetric Mean Absolute Percentage Error)
    def symmetric_mean_absolute_percentage_error(observed_values, predicted_values):
        return np.mean(2 * np.abs(observed_values - predicted_values) / (np.abs(observed_values) + np.abs(predicted_values))) * 100
    smape = symmetric_mean_absolute_percentage_error(observed_values, predicted_values)

    return rmse, mse, mae, mape, r2, mase, smape

import psutil
memoria_usada = []
def capturar_uso_memoria():
    # Captura o uso de memória RAM atual
    uso_memoria = psutil.virtual_memory().used
    memoria_usada.append(uso_memoria)

#LSD

In [5]:
df = pd.read_csv('https://docs.google.com/uc?export=download&id=11Wv1SFE_O0dGeYMWEXQpGOyd11Z2v41J')
df['Data'] = pd.to_datetime(df['Data'])
data = df['total_w']

In [6]:
import pandas as pd
import xarray as xr

# Supondo que você tenha um DataFrame chamado df com as colunas 'Data' e 'total_w'

# Primeiro, converta a coluna 'Data' para o tipo datetime
df['Data'] = pd.to_datetime(df['Data'])

# Em seguida, defina o índice do DataFrame como a coluna 'Data'
df.set_index('Data', inplace=True)

# Crie o objeto TimeSeries (DataArray) a partir do DataFrame
time_series = xr.DataArray(df['total_w'])

# O objeto time_series agora é um DataArray onde as datas são os índices.

In [7]:
train,test = split_train_test(df,'2023-07-19 23:00:00','2023-07-20 00:00:00')

In [ ]:
from darts.datasets import AirPassengersDataset
from darts.models import Prophet
from darts.utils.timeseries_generation import datetime_attribute_timeseries
series_ts = TimeSeries.from_series(train['total_w'])
series = series_ts
# optionally, use some future covariates; e.g. the value of the month encoded as a sine and cosine series
future_cov = datetime_attribute_timeseries(series, "hour", cyclic=True, add_length=test.shape[0])
# adding a seasonality (daily, weekly and yearly are included by default) and holidays
model = Prophet()
model.fit(series, future_covariates=future_cov)
capturar_uso_memoria()
pred = model.predict(future_covariates=future_cov,n=test.shape[0])
pred.values()

In [9]:
observed_values = test['total_w']
predicted_values = pred.values().flatten()
rmse, mse, mae, mape, r2, mase, smape = calculate_metrics(observed_values, predicted_values)
print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)
print("MAPE:", mape)
print("R²:", r2)
print("MASE:", mase)
print("sMAPE:", smape)
print('Memoria RAM:',max(memoria_usada))

RMSE: 2763.5508265112085
MSE: 7637213.170710783
MAE: 2077.1773177182868
MAPE: 9.351563182465462
R²: 0.41555919137999653
MASE: inf
sMAPE: 9.588550697502535
Memoria RAM: 1206579200


# CEEI

In [5]:
df = pd.read_csv('https://docs.google.com/uc?export=download&id=1-O64qB-YGpzpJ3QmRHTb6MxcqZYZRMdC')

df['Data'] = pd.to_datetime(df['Data'])
data = df['total_w']

In [6]:
import pandas as pd
import xarray as xr

# Supondo que você tenha um DataFrame chamado df com as colunas 'Data' e 'total_w'

# Primeiro, converta a coluna 'Data' para o tipo datetime
df['Data'] = pd.to_datetime(df['Data'])

# Em seguida, defina o índice do DataFrame como a coluna 'Data'
df.set_index('Data', inplace=True)

# Crie o objeto TimeSeries (DataArray) a partir do DataFrame
time_series = xr.DataArray(df['total_w'])

# O objeto time_series agora é um DataArray onde as datas são os índices.

In [7]:
train,test = split_train_test(df,'2023-07-31 23:00:00','2023-08-01 00:00:00')

In [8]:
from darts.datasets import AirPassengersDataset
from darts.models import Prophet
from darts.utils.timeseries_generation import datetime_attribute_timeseries
series_ts = TimeSeries.from_series(train['total_w'])
series = series_ts
# optionally, use some future covariates; e.g. the value of the month encoded as a sine and cosine series
future_cov = datetime_attribute_timeseries(series, "hour", cyclic=True, add_length=test.shape[0])
# adding a seasonality (daily, weekly and yearly are included by default) and holidays
model = Prophet()
model.fit(series, future_covariates=future_cov)
capturar_uso_memoria()
pred = model.predict(future_covariates=future_cov,n=test.shape[0])
pred.values()

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp9poiwf4s/w0qubgdz.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp9poiwf4s/midppmxa.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=29017', 'data', 'file=/tmp/tmp9poiwf4s/w0qubgdz.json', 'init=/tmp/tmp9poiwf4s/midppmxa.json', 'output', 'file=/tmp/tmp9poiwf4s/prophet_models3gkzko7/prophet_model-20240318025044.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
02:50:44 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
02:50:45 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


array([[1590.73228112],
       [1589.50242485],
       [1587.30301449],
       [1582.97694688],
       [1580.09185248],
       [1587.13884705],
       [1610.92239148],
       [1648.52343814],
       [1686.2942408 ],
       [1708.69419067],
       [1710.28087211],
       [1700.16413891],
       [1694.25206256],
       [1701.54875184],
       [1716.41489498],
       [1723.56614218],
       [1711.26781048],
       [1681.11961175],
       [1646.36769192],
       [1621.18196903],
       [1610.85580183],
       [1610.80605432],
       [1613.3630131 ],
       [1614.62118495],
       [1615.20089731],
       [1616.13106971],
       [1616.10586114],
       [1613.9410806 ],
       [1613.17769216],
       [1622.28091526],
       [1648.03041967],
       [1687.48378856],
       [1726.97162342],
       [1750.93364901],
       [1753.91000886],
       [1744.99554925],
       [1740.08594038],
       [1748.17563207],
       [1763.6184937 ],
       [1771.12625495],
       [1758.96322889],
       [1728.731

In [9]:
observed_values = test['total_w']
predicted_values = pred.values().flatten()
rmse, mse, mae, mape, r2, mase, smape = calculate_metrics(observed_values, predicted_values)
print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)
print("MAPE:", mape)
print("R²:", r2)
print("MASE:", mase)
print("sMAPE:", smape)
print('Memoria RAM:',max(memoria_usada))

RMSE: 522.9959948601922
MSE: 273524.8106398022
MAE: 460.4836405643124
MAPE: 37.03577996187222
R²: -0.007264118734812364
MASE: inf
sMAPE: 30.07127870737478
Memoria RAM: 1219051520


# CN

In [5]:
df = pd.read_csv('https://docs.google.com/uc?export=download&id=1-FpG1tqJgQJq0FVPozPvpzu-sO4n5zV3')

df['Data'] = pd.to_datetime(df['Data'])
data = df['total_w']

In [6]:
import pandas as pd
import xarray as xr

# Supondo que você tenha um DataFrame chamado df com as colunas 'Data' e 'total_w'

# Primeiro, converta a coluna 'Data' para o tipo datetime
df['Data'] = pd.to_datetime(df['Data'])

# Em seguida, defina o índice do DataFrame como a coluna 'Data'
df.set_index('Data', inplace=True)

# Crie o objeto TimeSeries (DataArray) a partir do DataFrame
time_series = xr.DataArray(df['total_w'])

# O objeto time_series agora é um DataArray onde as datas são os índices.

In [7]:
train,test = split_train_test(df,'2023-07-31 23:00:00','2023-08-01 00:00:00')

In [8]:
from darts.datasets import AirPassengersDataset
from darts.models import Prophet
from darts.utils.timeseries_generation import datetime_attribute_timeseries
series_ts = TimeSeries.from_series(train['total_w'])
series = series_ts
# optionally, use some future covariates; e.g. the value of the month encoded as a sine and cosine series
future_cov = datetime_attribute_timeseries(series, "hour", cyclic=True, add_length=test.shape[0])
# adding a seasonality (daily, weekly and yearly are included by default) and holidays
model = Prophet()
model.fit(series, future_covariates=future_cov)
capturar_uso_memoria()
pred = model.predict(future_covariates=future_cov,n=test.shape[0])
pred.values()

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp6s4_yfs_/0wul0ply.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp6s4_yfs_/8tr265sk.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=76947', 'data', 'file=/tmp/tmp6s4_yfs_/0wul0ply.json', 'init=/tmp/tmp6s4_yfs_/8tr265sk.json', 'output', 'file=/tmp/tmp6s4_yfs_/prophet_modelkhl9knln/prophet_model-20240318030133.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
03:01:33 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
03:01:34 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


array([[7872.90198412],
       [7875.01272441],
       [7856.67052884],
       [7822.65161125],
       [7802.585796  ],
       [7829.85257878],
       [7913.68683213],
       [8028.35571949],
       [8129.56302213],
       [8185.0039755 ],
       [8194.22090224],
       [8182.99920685],
       [8179.7079408 ],
       [8194.87909534],
       [8219.33938494],
       [8237.80544461],
       [8242.24629768],
       [8233.4063052 ],
       [8213.35397377],
       [8180.85133314],
       [8135.93567352],
       [8087.38925227],
       [8051.57297149],
       [8039.96993456],
       [8046.65251685],
       [8050.26205741],
       [8032.53590057],
       [7998.22908309],
       [7976.9609369 ],
       [8002.11139119],
       [8082.92678062],
       [8193.69672125],
       [8290.15825886],
       [8340.05038726],
       [8342.96922788],
       [8324.76344023],
       [8313.87407913],
       [8320.9130645 ],
       [8336.79403609],
       [8346.3265282 ],
       [8341.57733303],
       [8323.392

In [9]:
observed_values = test['total_w']
predicted_values = pred.values().flatten()
rmse, mse, mae, mape, r2, mase, smape = calculate_metrics(observed_values, predicted_values)
print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)
print("MAPE:", mape)
print("R²:", r2)
print("MASE:", mase)
print("sMAPE:", smape)
print('Memoria RAM:',max(memoria_usada))

RMSE: 1167.9328225022252
MSE: 1364067.0778780142
MAE: 973.8112929177918
MAPE: 13.65360303706043
R²: -1.0728284135183923
MASE: inf
sMAPE: 12.415126760929269
Memoria RAM: 1201786880


# SPLAB

In [5]:
df = pd.read_csv('https://docs.google.com/uc?export=download&id=1-H_5m5gkZuZd4Nx33LxOBI9cm0Rox8bR')

df['Data'] = pd.to_datetime(df['Data'])
data = df['total_w']

In [6]:
import pandas as pd
import xarray as xr

# Supondo que você tenha um DataFrame chamado df com as colunas 'Data' e 'total_w'

# Primeiro, converta a coluna 'Data' para o tipo datetime
df['Data'] = pd.to_datetime(df['Data'])

# Em seguida, defina o índice do DataFrame como a coluna 'Data'
df.set_index('Data', inplace=True)

# Crie o objeto TimeSeries (DataArray) a partir do DataFrame
time_series = xr.DataArray(df['total_w'])

In [7]:
train,test = split_train_test(df,'2023-07-31 23:00:00','2023-08-01 00:00:00')

In [8]:
from darts.datasets import AirPassengersDataset
from darts.models import Prophet
from darts.utils.timeseries_generation import datetime_attribute_timeseries
series_ts = TimeSeries.from_series(train['total_w'])
series = series_ts
# optionally, use some future covariates; e.g. the value of the month encoded as a sine and cosine series
future_cov = datetime_attribute_timeseries(series, "hour", cyclic=True, add_length=test.shape[0])
# adding a seasonality (daily, weekly and yearly are included by default) and holidays
model = Prophet()
model.fit(series, future_covariates=future_cov)
capturar_uso_memoria()
pred = model.predict(future_covariates=future_cov,n=test.shape[0])
pred.values()

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7diwsvc1/xjc5_mks.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7diwsvc1/a99w87l2.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=6343', 'data', 'file=/tmp/tmp7diwsvc1/xjc5_mks.json', 'init=/tmp/tmp7diwsvc1/a99w87l2.json', 'output', 'file=/tmp/tmp7diwsvc1/prophet_modelcc5ye68j/prophet_model-20240318030645.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
03:06:45 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
03:06:47 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


array([[2291.67053054],
       [2289.73508165],
       [2286.49042628],
       [2284.88351918],
       [2288.16964199],
       [2297.12809697],
       [2309.15730012],
       [2319.99232949],
       [2326.58845807],
       [2328.86057652],
       [2329.11931884],
       [2330.02731434],
       [2332.89976683],
       [2337.45789684],
       [2342.63849069],
       [2347.30708682],
       [2350.3074614 ],
       [2350.35083433],
       [2346.51136883],
       [2339.23743962],
       [2330.81682771],
       [2324.37872262],
       [2321.85029664],
       [2322.53026101],
       [2323.71505088],
       [2323.09952806],
       [2320.98464566],
       [2320.30617434],
       [2324.31028666],
       [2333.76936112],
       [2346.07716017],
       [2356.96643709],
       [2363.39250028],
       [2365.27263994],
       [2364.92222992],
       [2365.01092851],
       [2366.86317979],
       [2370.21155191],
       [2374.00615771],
       [2377.12768949],
       [2378.43673156],
       [2376.662

In [9]:
observed_values = test['total_w']
predicted_values = pred.values().flatten()
rmse, mse, mae, mape, r2, mase, smape = calculate_metrics(observed_values, predicted_values)
print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)
print("MAPE:", mape)
print("R²:", r2)
print("MASE:", mase)
print("sMAPE:", smape)
print('Memoria RAM:',max(memoria_usada))

RMSE: 386.3551938629281
MSE: 149270.33582486075
MAE: 305.4770614862365
MAPE: 16.088296612412258
R²: -1.2065145574900034
MASE: inf
sMAPE: 14.157970540939772
Memoria RAM: 1216733184


# CTLABS

In [5]:
df = pd.read_csv('https://docs.google.com/uc?export=download&id=1-F71fdeBhScnn01Ev78BGc-ZLGr_WHJR')

df['Data'] = pd.to_datetime(df['Data'])
data = df['total_w']

In [6]:
import pandas as pd
import xarray as xr

# Supondo que você tenha um DataFrame chamado df com as colunas 'Data' e 'total_w'

# Primeiro, converta a coluna 'Data' para o tipo datetime
df['Data'] = pd.to_datetime(df['Data'])

# Em seguida, defina o índice do DataFrame como a coluna 'Data'
df.set_index('Data', inplace=True)

# Crie o objeto TimeSeries (DataArray) a partir do DataFrame
time_series = xr.DataArray(df['total_w'])

In [7]:
train,test = split_train_test(df,'2023-07-31 23:00:00','2023-08-01 00:00:00')

In [8]:
from darts.datasets import AirPassengersDataset
from darts.models import Prophet
from darts.utils.timeseries_generation import datetime_attribute_timeseries
series_ts = TimeSeries.from_series(train['total_w'])
series = series_ts
# optionally, use some future covariates; e.g. the value of the month encoded as a sine and cosine series
future_cov = datetime_attribute_timeseries(series, "hour", cyclic=True, add_length=test.shape[0])
# adding a seasonality (daily, weekly and yearly are included by default) and holidays
model = Prophet()
model.fit(series, future_covariates=future_cov)
capturar_uso_memoria()
pred = model.predict(future_covariates=future_cov,n=test.shape[0])
pred.values()

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpe1pyxuw5/7ep2szao.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpe1pyxuw5/xpyioclv.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=54662', 'data', 'file=/tmp/tmpe1pyxuw5/7ep2szao.json', 'init=/tmp/tmpe1pyxuw5/xpyioclv.json', 'output', 'file=/tmp/tmpe1pyxuw5/prophet_modelqnz_7gb9/prophet_model-20240318031306.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
03:13:06 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
03:13:07 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


array([[ 953.26991253],
       [ 954.81548928],
       [ 954.90938638],
       [ 951.49520426],
       [ 946.45757177],
       [ 946.26394141],
       [ 957.66495308],
       [ 981.3392713 ],
       [1009.36896111],
       [1029.82331955],
       [1035.41708968],
       [1029.05183521],
       [1021.26361549],
       [1021.50623521],
       [1030.58578609],
       [1040.48991736],
       [1041.33588654],
       [1029.18378356],
       [1008.31678534],
       [ 986.98677629],
       [ 971.23590227],
       [ 962.15878052],
       [ 957.80163222],
       [ 956.40690835],
       [ 957.31541224],
       [ 959.38198623],
       [ 960.0030433 ],
       [ 957.11091747],
       [ 952.57920337],
       [ 952.8647097 ],
       [ 964.70797426],
       [ 988.77824368],
       [1017.14898129],
       [1037.88181808],
       [1043.68487228],
       [1037.45421437],
       [1029.72161582],
       [1029.93785525],
       [1038.90729851],
       [1048.61718889],
       [1049.18569888],
       [1036.675

In [9]:
observed_values = test['total_w']
predicted_values = pred.values().flatten()
rmse, mse, mae, mape, r2, mase, smape = calculate_metrics(observed_values, predicted_values)
print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)
print("MAPE:", mape)
print("R²:", r2)
print("MASE:", mase)
print("sMAPE:", smape)
print('Memoria RAM:',max(memoria_usada))

RMSE: 338.8210187790634
MSE: 114799.68276648247
MAE: 261.3707849173855
MAPE: 21.584967385766006
R²: -0.14192244157346523
MASE: inf
sMAPE: 23.331737112525055
Memoria RAM: 1192837120
